# Using Qiskit to solve a linear system of equations
**Note: This goes with the handwritten notes found in: Handwritten_notes/5_QHHL_Algorithm_for_Linear_systems.pdf.   
Please refer to that for clarification.**  
This notebook applies the HHL quantum algorithm (Harrow-Hassidim-Lloyd) to solve a system of linear equations using qiskit-aqua.

In [8]:
import warnings
warnings.filterwarnings('ignore') # ignore depreciation warnings 

from qiskit.aqua import run_algorithm
from qiskit.aqua.input import LinearSystemInput
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms.classical import ExactLSsolver
import numpy as np

In [2]:
# set the initial parameters:
base_parameters = {
    'problem' :
    {
        'name': 'linear_system'
    },
    'algorithm':
    {
        'name': 'HHL'
    },
    'eigs':
    {
        'expansion_mode' : 'suzuki',
        'expansion_order': 2,
        'name': 'EigsQPE',
        'num_ancillae' : 3,
        'num_time_slices': 50
    },
    'reciprocal':
    {
        'name' : 'Lookup'
    },
    'backend':
    {
        'provider' : 'qiskit.BasicAer',
        'name' : 'statevector_simulator'
    }
}

In [3]:
# function to calculate and print fidelity
def fidelity(hhl,ref):
    # normalized HHL and reference solutions
    solution_hhl_normed = hhl/np.linalg.norm(hhl)
    solution_ref_normed = ref/np.linalg.norm(ref)
    fidelity = state_fidelity(solution_hhl_normed, solution_ref_normed )
    print(f"Fidelity = {fidelity}") # using python 3.6+

## Example 1 - Diagonal Matrix:
Input matrix:  
![A-matrix](https://render.githubusercontent.com/render/math?math=A%3D%0A%5Cbegin%7Bbmatrix%7D%0A1%20%26amp%3B%200%20%5C%5C%0A0%20%26amp%3B%202%0A%5Cend%7Bbmatrix%7D&mode=display)  
unit vector:  

![b-vector](https://render.githubusercontent.com/render/math?math=%5Cvec%7Bb%7D%3D%20%5Cleft%28%20%5Cbegin%7Barray%7D%7Bc%7D1%20%5C%5C%204%20%20%5Cend%7Barray%7D%20%5Cright%29&mode=display)  

**Note:** the result dictionary contains several return values. the HHL solution for![x](https://render.githubusercontent.com/render/math?math=%5Cvec%7Bx%7D&mode=inline)is accessible by the key `'solution'`. 
The classical solution of the linear system of equations is calculated using standard linear algebra functions in numpy. The fidelity between the HHl solution and classical solution is also given in the output. The probability with which HHL is running successfully is also given. (As written in the notes, it is successful when the HHL ancillary qubit is |1>).

In [5]:
A_matrix = [[1, 0],[0, 2]]
b_vector = [1,4]
base_parameters['input'] = {
    'name' : 'LinearSystemInput',
    'matrix' : A_matrix,
    'vector' : b_vector,
}

In [9]:
HHL_result = run_algorithm(base_parameters) # returns the result dictionary discussed above
ref_result = ExactLSsolver(A_matrix,b_vector).run() # returns classical solution from numpy

In [10]:
# now we can print out results and compare it to what we get from classical means.
HHL_solution = np.round(HHL_result['solution'], 5)
ref_solution = np.round(ref_result['solution'], 5)

print(f"HHL quantum Algorithm solution = {HHL_solution}")
print(f"Classical Linear System solution = {ref_solution}\n")
print(f"Probability = {HHL_result['probability_result']}")
fidelity(HHL_solution, ref_solution)   

HHL quantum Algorithm solution = [1.05859+0.j 1.99245+0.j]
Classical Linear System solution = [1. 2.]

Probability = 0.024629684664842826
Fidelity = 0.9993887568912113


In [11]:
# scale the probability, and check for changes 
mod_parameters = base_parameters # modified paramters
mod_parameters['reciprocal'] = {
    'scale' : 0.5
}

HHL_result = run_algorithm(mod_parameters)
ref_result = ExactLSsolver(A_matrix, b_vector).run()

HHL_solution = np.round(HHL_result['solution'], 5)
ref_solution = np.round(ref_result['solution'], 5)

print(f"HHL quantum Algorithm solution = {HHL_solution}")
print(f"Classical Linear System solution = {ref_solution}\n")
print(f"Probability = {HHL_result['probability_result']}")
fidelity(HHL_solution, ref_solution)   

HHL quantum Algorithm solution = [0.84664+0.j 2.01762+0.j]
Classical Linear System solution = [1. 2.]

Probability = 0.36143730037074434
Fidelity = 0.9956054532236346


In [13]:
# investigating the quantum circuit features
# circuit width (how many qubits are required)
print(f"circuit width = {HHL_result['circuit_info']['width']}")
# circuit depth (maximum number of gates applied to one qubit)
print(f"circuit_depth = {HHL_result['circuit_info']['depth']}")

circuit width = 7
circuit_depth = 12256


## Example 2 - Non-Diagonal Matrix:
Matrix:  
![A-matrix](https://render.githubusercontent.com/render/math?math=A%3D%0A%5Cbegin%7Bbmatrix%7D%0A1%20%26amp%3B%203%20%5C%5C%0A3%20%26amp%3B%202%0A%5Cend%7Bbmatrix%7D&mode=display)  
Unit Vector:
![b-vector](https://render.githubusercontent.com/render/math?math=%5Cvec%7Bb%7D%3D%20%5Cleft%28%20%5Cbegin%7Barray%7D%7Bc%7D1%20%5C%5C%201%20%20%5Cend%7Barray%7D%20%5Cright%29&mode=display)  


In [20]:
A_matrix = [[1,3],[3,2]]
b_vector = [1,1]

# declare the parameters
ex2_parameters = base_parameters
ex2_parameters['input'] = {
    'name' : 'LinearSystemInput',
    'matrix' : A_matrix,
    'vector' : b_vector
}
ex2_parameters['reciprocal'] = {
    'negative_evals' : True
}
ex2_parameters['eigs'] = {
    'negative_evals' : True
}

In [22]:
HHL_result = run_algorithm(ex2_parameters)
ref_result = ExactLSsolver(A_matrix, b_vector).run()

HHL_solution = np.round(HHL_result['solution'], 5)
ref_solution = np.round(ref_result['solution'], 5)

print(f"(ex2) HHL quantum Algorithm solution = {HHL_solution}")
print(f"(ex2) Classical Linear System solution = {ref_solution}\n")
print(f"(ex2) Probability = {HHL_result['probability_result']}")
fidelity(HHL_solution, ref_solution)

(ex2) HHL quantum Algorithm solution = [0.14223-5.e-05j 0.28622+7.e-05j]
(ex2) Classical Linear System solution = [0.14286 0.28571]

(ex2) Probability = 0.00031603500306951664
Fidelity = 0.9999938095340438


In [25]:
# note that the circuit depth is increased by 6 times compared to the first example.
print(f"circuit width = {HHL_result['circuit_info']['width']}")
print(f"circuit depth = {HHL_result['circuit_info']['depth']}")

circuit width = 11
circuit depth = 73313


##  Example 3 - 8x8 Non-Diagonal Matrix:
Matrix:  
![A-matrix](https://render.githubusercontent.com/render/math?math=A%3D%0A%5Cbegin%7Bbmatrix%7D%0A4%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%201%20%5C%5C%0A0%20%26amp%3B%203%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%5C%5C%0A0%20%26amp%3B%200%20%26amp%3B%208%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%5C%5C%0A0%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%205%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%5C%5C%0A0%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%202%20%26amp%3B%201%20%26amp%3B%200%20%26amp%3B%200%20%5C%5C%0A0%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%201%20%26amp%3B%201%20%26amp%3B%200%20%26amp%3B%200%20%5C%5C%0A0%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%201%20%26amp%3B%200%20%5C%5C%0A1%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%200%20%26amp%3B%205%0A%5Cend%7Bbmatrix%7D&mode=display)  
Unit vector:
![b-vector](https://render.githubusercontent.com/render/math?math=%5Cvec%7Bb%7D%3D%20%5Cleft%28%20%5Cbegin%7Barray%7D%7Bc%7D1%20%5C%5C%200%20%5C%5C%200%20%5C%5C%200%20%5C%5C%200%20%5C%5C%200%20%5C%5C%200%20%5C%5C%201%20%5Cend%7Barray%7D%20%5Cright%29&mode=display)

In [27]:
A_matrix = [[4, 0, 0, 0, 0, 0, 0, 1],
          [0, 3, 0, 0, 0, 0, 0, 0],
          [0, 0, 8, 0, 0, 0, 0, 0],
          [0, 0, 0, 5, 0, 0, 0, 0],
          [0, 0, 0, 0, 2, 1, 0, 0],
          [0, 0, 0, 0, 1, 1, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0],
          [1, 0, 0, 0, 0, 0, 0, 5]]
b_vector = [1, 0, 0, 0, 0, 0, 0, 1]
ex3_parameters = base_parameters
ex3_parameters['input'] = {
    'name': 'LinearSystemInput',
    'matrix': A_matrix,
    'vector': b_vector
}

In [31]:
#HHL_result = run_algorithm(ex3_parameters)
#ref_result = ExactLSsolver(A_matrix, b_vector).run()

#HHL_solution = np.round(HHL_result['solution'], 5)
#ref_solution = np.round(ref_result['solution'], 5)

#print(f"(ex3) HHL quantum Algorithm solution = {HHL_solution}")
#print(f"(ex3) Classical Linear System solution = {ref_solution}\n")
#print(f"(ex3) Probability = {HHL_result['probability_result']}")
#fidelity(HHL_solution, ref_solution)

Above is the code to solve the 8x8 non-diagonal matrix. Since my computer is too slow (it takes too long to for the quantum simulation), you may want to download this notebook and try it on your own machine.